<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/function%20documentation%20generation/go/small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the library and download the pretrained models

In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5==0.6.4

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

!wget "https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1" -O vocabulary.zip
!unzip vocabulary.zip
!rm vocabulary.zip
!wget "https://www.dropbox.com/sh/uhpf2fc6b9x217q/AABB4_apYAEVg_1XmxIZYV2za?dl=1" -O go.zip
!unzip go.zip
!rm go.zip

Installing dependencies...
     |████████████████████████████████| 163kB 2.8MB/s 
     |████████████████████████████████| 1.3MB 9.2MB/s 
     |████████████████████████████████| 2.6MB 13.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 348kB 37.7MB/s 
     |████████████████████████████████| 1.1MB 34.1MB/s 
     |████████████████████████████████| 3.6MB 47.6MB/s 
     |████████████████████████████████| 2.9MB 48.8MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
--2020-11-10 09:56:08--  https://www.dropbox.com/sh/kjoqdpj7e16dny9/AADdvjWVFckCgNQN-AqMKhiDa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/kjoqdpj7e16d

## Set sentencepiece model

In [ ]:
from t5.data.sentencepiece_vocabulary import SentencePieceVocabulary

vocab_model_path = 'code_spm_unigram_40M.model'
vocab = SentencePieceVocabulary(vocab_model_path, extra_ids=100)

print("Vocab has a size of %d\n" % vocab.vocab_size)

Vocab has a size of 32100



## Set the preprocessors and the task registry for the t5 model

In [ ]:
def go_codeSearchNet_dataset_fn(split, shuffle_files=False):
    del shuffle_files

    ds = tf.data.TextLineDataset(go_path[split])
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""], field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    ds = ds.map(lambda *ex: dict(zip(["code", "docstring"], ex)))
    return ds


def go_preprocessor(ds):
    def normalize_text(text):
        return text

    def to_inputs_and_targets(ex):
        return {
            "inputs": tf.strings.join(["function documentation generation go: ", normalize_text(ex["code"])]),
            "targets": normalize_text(ex["docstring"])
        }

    return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)


t5.data.TaskRegistry.remove('function_documentation_generation_go_code')
t5.data.TaskRegistry.add(
    "function_documentation_generation_go_code",
    dataset_fn=go_codeSearchNet_dataset_fn,
    output_features={
        "inputs": t5.data.utils.Feature(vocabulary=vocab),
        "targets": t5.data.utils.Feature(vocabulary=vocab),
    },
    splits=["train", "validation"],
    text_preprocessor=[go_preprocessor],
    postprocess_fn=t5.data.postprocessors.lower_text,
    metric_fns=[t5.evaluation.metrics.bleu, t5.evaluation.metrics.accuracy, t5.evaluation.metrics.rouge],
)

## Set t5 small model

In [ ]:
MODEL_DIR = "small"
model_parallelism = 1
train_batch_size = 256

tf.io.gfile.makedirs(MODEL_DIR)
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=None,
    tpu_topology=None,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    mesh_shape="model:1,batch:1",
    mesh_devices=["GPU:0"],
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=None,
    iterations_per_loop=100,
)

## Code Documentation Summarization

### Give the code for summarization

In [16]:
code = "func (pr *Progress) needSnapshotAbort() bool {\n\treturn pr.State == ProgressStateSnapshot && pr.Match >= pr.PendingSnapshot\n}" #@param {type:"raw"}


### Parsing and Tokenization

In [3]:
!pip install tree_sitter
!git clone https://github.com/tree-sitter/tree-sitter-go

     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for tree-sitter: filename=tree_sitter-0.2.0-cp36-cp36m-linux_x86_64.whl size=297391 sha256=aa1d47f83a8a3094885e3ff8f19b73da2b55c325b770de79cb48821565c79c86
  Stored in directory: /root/.cache/pip/wheels/3b/a6/01/2290cc8441301a07e7717e9e03c6bbc0388f71c6bf1f7f37c1
Successfully built tree-sitter
Cloning into 'tree-sitter-go'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 1396 (delta 25), reused 47 (delta 16), pack-reused 1328
Receiving objects: 100% (1396/1396), 7.46 MiB | 22.93 MiB/s, done.
Resolving deltas: 100% (903/903), done.


In [4]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-go']
)

GO_LANGUAGE = Language('build/my-languages.so', 'go')
parser = Parser()
parser.set_language(GO_LANGUAGE)

In [14]:
def get_string_from_code(node, lines):
  line_start = node.start_point[0]
  line_end = node.end_point[0]
  char_start = node.start_point[1]
  char_end = node.end_point[1]
  if line_start != line_end:
    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
  else:
    code_list.append(lines[line_start][char_start:char_end])

def my_traverse(node, code_list):
  lines = code.split('\n')
  if node.child_count == 0:
    get_string_from_code(node, lines)
  elif node.type == 'string':
    get_string_from_code(node, lines)
  else:
    for n in node.children:
      my_traverse(n, code_list)
 
  return ' '.join(code_list)

In [31]:
tree = parser.parse(bytes(code, "utf8"))
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: func ( pr * Progress ) needSnapshotAbort ( ) bool { return pr . State == ProgressStateSnapshot && pr . Match >= pr . PendingSnapshot   }


### Record the code for summarization with the prefix to a txt file

In [ ]:
codes = [tokenized_code]

inputs_path = 'input.txt'
with tf.io.gfile.GFile(inputs_path, "w") as f:
  for c in codes:
    f.write("function documentation generation go: %s\n" % c)

predict_outputs_path = 'MtfModel-output.txt'


### Running the model with the best checkpoint to summarize the given code

In [ ]:
model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
model.predict(
    input_file="input.txt",
    output_file=predict_outputs_path,
    checkpoint_steps=5000,
    beam_size=4,
    vocabulary=vocab, 
    # Select the most probable output token at each step.
    temperature=0,
)

INFO:tensorflow:Using config: {'_model_dir': 'small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_s

### Code Summarization Result

In [ ]:
prediction_file = "MtfModel-output.txt-5000"
print("\nPredictions using checkpoint 5000:\n" )
with tf.io.gfile.GFile(prediction_file) as f:
  for c, d in zip(codes, f):
    if c:
      print("Code for prediction: " + c + '\n')
      print("Generated Documentation: " + d)



Predictions using checkpoint 5000:

Code for prediction: func ( pr * Progress ) needSnapshotAbort ( ) bool { return pr . State == ProgressStateSnapshot && pr . Match >= pr . PendingSnapshot   }

Generated Documentation: b'needSnapshotAbort returns true if we need to roll a snapshot .'

